In [ ]:
import os
import numpy as np
import shutil
import pandas as pd

# Создание директорий под тренировочный и валидационный сеты
root_dir = 'internship_data'
maleCls = '/male'
femaleCls = '/female'

os.makedirs(root_dir +'/train' + maleCls)
os.makedirs(root_dir +'/train' + femaleCls)
os.makedirs(root_dir +'/val' + maleCls)
os.makedirs(root_dir +'/val' + femaleCls)

src1 = root_dir + maleCls
src2 = root_dir + femaleCls

#Получение списка с названиями всех изображений 
allFileNames1 = os.listdir(src1)
allFileNames2 = os.listdir(src2)
np.random.shuffle(allFileNames1)
np.random.shuffle(allFileNames2)
# Разбиение на 2 сета
train_FileNames1, val_FileNames1 = np.split(np.array(allFileNames1), [int(len(allFileNames1)*0.9)])
train_FileNames2, val_FileNames2 = np.split(np.array(allFileNames2), [int(len(allFileNames2)*0.9)])

train_FileNames1 = [src1+'/'+ name for name in train_FileNames1.tolist()]
val_FileNames1 = [src1+'/' + name for name in val_FileNames1.tolist()]
train_FileNames2 = [src2+'/'+ name for name in train_FileNames2.tolist()]
val_FileNames2 = [src2+'/' + name for name in val_FileNames2.tolist()]

print('Total images: MaleCls: ',len(allFileNames1), ' FemaleCls: ', len(allFileNames2))
print('Training: MaleCls: ',len(train_FileNames1), ' FemaleCls: ', len(train_FileNames2))
print('Validation: MaleCls: ',len(val_FileNames1), ' FemaleCls: ', len(val_FileNames2))
# Копирование изображений в папки
for name in train_FileNames1:
    shutil.copy(name, "internship_data/train"+maleCls)

for name in val_FileNames1:
    shutil.copy(name, "internship_data/val"+maleCls)

for name in train_FileNames2:
    shutil.copy(name, "internship_data/train"+femaleCls)

for name in val_FileNames2:
    shutil.copy(name, "internship_data/val"+femaleCls)

# Создание csv файлов
mtrain_names = ['male/' + name for name in os.listdir('internship_data/train/male/')]
ftrain_names = ['female/' + name for name in os.listdir('internship_data/train/female/')]

train_data_m = pd.DataFrame({'image_name': mtrain_names, 'class': np.zeros(len(mtrain_names), dtype=int)})
train_data_f = pd.DataFrame({'image_name': ftrain_names, 'class': np.ones(len(ftrain_names), dtype=int)})
train_data = pd.concat([train_data_m, train_data_f], ignore_index=True)
train_data = train_data.sample(frac = 1)
train_data.to_csv('train_data.csv', index = False)


mval_names = ['male/' + name for name in os.listdir('internship_data/val/male/')]
fval_names = ['female/' + name for name in os.listdir('internship_data/val/female/')]

val_data_m = pd.DataFrame({'image_name': mval_names, 'class': np.zeros(len(mval_names), dtype=int)})
val_data_f = pd.DataFrame({'image_name': fval_names, 'class': np.ones(len(fval_names), dtype=int)})
val_data = pd.concat([val_data_m, val_data_f], ignore_index=True)
val_data = val_data.sample(frac = 1)
val_data.to_csv('val_data.csv', index = False)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms, utils
import matplotlib.pyplot as plt
import time
import os
import copy
from __future__ import print_function, division
import pandas as pd
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader

# Класс для кастомного датасета
class FaceRecognitionDataset(Dataset):
  # Переопредленный конструктор класса
  def __init__(self, csv_file, root_dir, transform=None):
    self.source = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform
  # Метод получения размера
  def __len__(self):
    return(len(self.source))
  # Метод получения очередного элемента датасета
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()

    img_name = os.path.join(self.root_dir, self.source.iloc[idx, 0])
    image = io.imread(img_name)
    clas = self.source.iloc[idx, 1]
    sample = {'image': image, 'class': clas}

    if self.transform:
      sample = self.transform(sample)

    return sample

# Класс для изменения размера изображения для кастомного датасета
class Rescale(object):

    def __init__(self, output_size):
      assert isinstance(output_size, (int, tuple))
      self.output_size = output_size

    def __call__(self, sample):
      image, clas = sample['image'], sample['class']

      h, w = image.shape[:2]
      if isinstance(self.output_size, int):
        if h > w:
          new_h, new_w = self.output_size * h / w, self.output_size
        else:
          new_h, new_w = self.output_size, self.output_size * w / h
      else:
        new_h, new_w = self.output_size

      new_h, new_w = int(new_h), int(new_w)
      img = transform.resize(image, (new_h, new_w))
      return {'image': img, 'class': clas}


# Класс для перевода изображения, хранящегося как numpy массив
# в pytorch тензор, так как они имеют разные структуры размещения данных 
class ToTensor(object):

  def __call__(self, sample):
    image, clas = sample['image'], sample['class']
    image = image.transpose((2, 0, 1))
    return {'image': torch.from_numpy(image).float(), 'class': clas}

# Класс для нормализации изображениий, так как используемые модели
# были обучены на изображениях с указанными показателями нормализации
class Normalize():

  def __call__(self, sample):
    image, clas = sample['image'], sample['class']
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    image = transforms.functional.normalize(image, mean=mean, std=std)
    return {'image': image, 'class': clas}

In [3]:
# Функция сохранения чекпоинтов модели
def save_checkpoint(epoch, model, optimizer):

    state = {'epoch': epoch,
             'model': model,
             'optimizer': optimizer}
    filename = 'checkpoint_resnet50.pth.tar'
    torch.save(state, filename)

# Выбор девайса на котором будет производиться обучение,
# выбирается GPU, если доступен, иначе CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Количество классов в датасете (пол: мужской, женский)
num_classes = 2

# Размер партии изображений для обучения
batch_size = 16

# Количесвто эпох для обучения
num_epochs = 3

# Флаг для использования feature extracting 
# (Переобучение параметров только измененных слоев нейросети). Если установлен False,
# применяется finetune (переобучение параметров всех слоев нейросети)
feature_extract = False

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()
    # Список для хранения значений точности модели на валидационном наборе,
    # подсчитанные после окончания каждой эпохи обучения
    val_acc_history = []
    # Переменная для выбора лучшей модели по итогам прохождения всех эпох
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    # Главный цикл обучения
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Каждая эпоха делится на 2 фазы: обучение и валидация
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # установка модели в режим обучения
            else:
                model.eval()   # установка модели в режим валидации
            # Текущие потери
            running_loss = 0.0
            running_corrects = 0

            # Проход по всем изображениям, подаваемым загрузчиком партиями
            for sample in dataloaders[phase]:
                inputs = sample['image']
                labels = sample['class']
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Обнуление градиентов перед каждым обратным распространением
                # так как по умолчанию градиенты аккумулируются
                optimizer.zero_grad()
 
                # Сохранение истории только при обучении
                with torch.set_grad_enabled(phase == 'train'):
                    # Получение предсказаний модели и расчет потерь
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # Обратное распространение и оптимизация при обучении
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Запись статистики
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # Сохранение весов модели после очередной эпохи обучения, если она точнее
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
            # Сохранение чекпоинта
            save_checkpoint(epoch, model, optimizer)
            # Очищение переменных
            del preds, outputs, inputs, labels

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # Загрузка лучших весов
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
# Функция для feature extracting для выключения просчета градиентов у параметров, которые не будут обучаться
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

# Функция инициализации модели
def initialize_model(num_classes, feature_extract, use_pretrained=True):
  # в параметрах указывается использование предобученных весов
  model_ft = models.resnet50(pretrained=use_pretrained)
  set_parameter_requires_grad(model_ft, feature_extract)
  num_ftrs = model_ft.fc.in_features
  # Изменение последного решающего fc слоя, для того чтобы выход нейросети был равен указанному количеству классов
  model_ft.fc = nn.Linear(num_ftrs, num_classes)
  # Размер входных изображений
  input_size = 224

  return model_ft, input_size

# Инициализация моедели
model_ft, input_size = initialize_model(num_classes, feature_extract, use_pretrained=True)

In [5]:
# Датасет для обучения
train_dataset = FaceRecognitionDataset(csv_file='train_data.csv', root_dir='internship_data/train/',
                                             transform=transforms.Compose([
                                              Rescale((224, 224)),
                                              ToTensor(),
                                              Normalize()]))
# Датасет для валидации
val_dataset = FaceRecognitionDataset(csv_file='val_data.csv', root_dir='internship_data/val/',
                                             transform=transforms.Compose([
                                              Rescale((224, 224)),
                                              ToTensor(),
                                              Normalize()]))
# Загрузчики данных
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

dataloaders_dict = {'train': train_loader, 'val': val_loader}

In [ ]:
# Переносим модель на выбранный девайс
model_ft = model_ft.to(device)

# Параметры которые будут оптимизироваться
params_to_update = model_ft.parameters()
# Оптимизатор
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

# Функция подсчета потерь для валидации
criterion = nn.CrossEntropyLoss()

# Запуск тренировки модели
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)